# Membaca data dan membuat skema data

Pertama kita akan membaca data ke DataFrame dan membuat skema dari data kita, persis seperti yang telah kita lakukan sebelumnya. Kita akan membaca data flight dan data airport.

In [1]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession

#membuat session, untuk mengakses semua fungsi spark dan DataFrame API
appName = "Pengenalan DataFrame Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])

flights = spark.read.csv('dataset/raw-flight-data.csv', 
                         schema=flightSchema, header=True)
flights.show(2)

AnalysisException: Path does not exist: file:/C:/Users/User/Documents/python/dataset/raw-flight-data.csv

In [ ]:
airportSchema = StructType([
  StructField("airport_id", IntegerType(), False),
  StructField("city", StringType(), False),
  StructField("state", StringType(), False),
  StructField("name", StringType(), False),
])

airports = spark.read.csv('dataset/airports.csv', header=True, 
                          schema=airportSchema)
airports.show(2)

Kita dapat menggabungkan dua DataFrame (flights dan aiports) jika kita perlukan. Hal ini mirip dengan pemrograman SQL di relational database.

In [ ]:
flightsByOrigin = flights.join(airports,
                               flights.OriginAirportID == 
                               airports.airport_id).groupBy("city").count()
flightsByOrigin.show(3)

# Menghadapi data duplikat


Data preprocessing pertama adalah mengecek apakah data kita ada yang terduplikat atau tidak. Jika ada data yang terduplikat, kita dapat menghapusnya dengan fungsi "dropDuplicates()" pada DataFrame kita. Kode di bawah ini adalah membuat DataFrame dimana baris data duplikat sudah kita hapus. Untuk menghitung berapa jumlah baris data duplikat, kita dapat menghitungnya dengan cara jumlah baris data asli dikurangi dengan jumlah baris setelah kita hapus duplikatnya.

In [ ]:
jumlahBarisDataFlight = flights.count(); 
print("jumlah baris data flight: ", jumlahBarisDataFlight)
flightHapusDuplikat = flights.dropDuplicates(); 
print("jumlah baris data flight tanpa duplikat: ", flightHapusDuplikat.count())
dataDuplikat = jumlahBarisDataFlight - flightHapusDuplikat.count(); 
print("jumlah baris data duplikat: ", dataDuplikat)

Kita juga dapat menspesifikasi pengecekan duplikatnya khusus pada kolom tertentu.

In [ ]:
from pyspark.sql import Row
sc = SparkContext.getOrCreate()
df = sc.parallelize([ \
     Row(nama='Roni', umur=27, tinggi=168), \
     Row(nama='Roni', umur=5, tinggi=165), \
     Row(nama='Roni', umur=27, tinggi=168)]).toDF()
df.show()
df.dropDuplicates().show()

Mungkin akan terjadi error saat menjalankan "sc.parallelize" di atas, kita dapat mengatasinya dengan cara klik "Kernel>Restart". Menghapus duplikat berdasarkan kolom tertentu dapat kita gunakan misalkan kita tidak ingin ada duplikat data dengan ID yang sama. Kode di bawah ini adalah contoh pengecekan duplikat data berdasarkan kolom "nama".

In [ ]:
df.dropDuplicates(['nama']).show()

# Menghadapi missing value (data yang tidak lengkap/ada)

Bersamaan dengan mendeteksi jika ada data yang terduplikat, kita juga dapat mendeteksi jika ada missing value. Kita dapat menghapus baris data yang terdapat missing value tersebut, atau mengisi missing value tersebut dengan nilai lain, misalnya nilai rata-rata dari kolom tersebut. Fungsi "dorpna" di bawah ini berguna untuk membuat DataFrame baru dimana baris data dengan missing value sudah terhapus. Kita dapat menentukan subset dari kolomnya yang akan kita cek missing value atau tidak, juga kita dapat menentukan apakah menghapus baris datanya ketika semua kolom tersebut missing value atau setidaknya ada satu missing value saja. Kode di bawah ini akan menghapus baris data jika setidaknya ada salah satu missing value di kolom "ArrDelay" dan "DepDelay". Kemudian kita akan menghitung berapa jumlah data yang terdapat missing value dengan cara mengurangi jumlah baris di data awal dikurangi dengan jumlah baris setelah data kita hapus saat terdapat missing value.

In [ ]:
flightsNoMissingValue = flights.dropDuplicates().dropna(
    how="any", subset=["ArrDelay", "DepDelay"])
jumlahBarisMissingValue = jumlahBarisDataFlight - flightsNoMissingValue.count()
print("jumlah baris data missing value: ", jumlahBarisMissingValue)

Menghapus data yang kosong terkadang merupakan pilihan yang kurang baik. Kita dapat menggunakan pilihan lain, mengisi data yang kosong dengan nilai reratanya misalnya. Berikut ini adalah kodenya. Adapun jumlah datanya adalah sama dengan jumlah baris data saat kita melakukan penghapusan data duplikat yang telah kita lakukan di atas.

In [ ]:
meanArrDelay = flights.groupBy().avg("ArrDelay").take(1)[0][0]
print("mean ArrDelay: ", meanArrDelay)
meanDepDelay = flights.groupBy().avg("DepDelay").take(1)[0][0]
print("mean DepDelay: ", meanDepDelay)
flightsCleanData=flights.dropDuplicates().fillna(
    {'ArrDelay': meanArrDelay, 'DepDelay': meanDepDelay})
print("jumlah baris clean data flight: ", flightsCleanData.count())

Untuk menghitung nilai mean (rerata) di atas, kodenya agak kompleks dan mungkin kurang 'friendly' bagi pendatang baru Spark. Adapun menurut penilaian penulis, API DataFrame dari Spark memang tidak sefleksibel dan sekaya API DataFrame dari pandas python. Untuk menghitung mean dari suatu kolom di DataFrame Spark, kita dapat melakukannya dengan kode "DataFrame_kita.groupBy().avg("kolom_yang_ingin_kita_hitung")". Akan tetapi dari kode ini kita mendapatkan data berupa DataFrame, sedangkan yang kita inginkan adalah nilainya langsung dalam float. Kita dapat mengakses isi DataFrame tersebut dengan mengambil baris pertama dengan perintah "take(1)", dan nilainya dalam bentuk list matrik baris-kolom. Selanjutnya kita akses nilai meannya yaitu di baris 1 dan kolom 1 dengan kode "[0][0]" (indexing dimulai dari 0).

# Mengeksplorasi statistik data

Kita dapat mengeksplorasi statistik dari data kita seperti nilai mean, standard deviasi, maksimal dan minimal dari setiap kolom data kita dengan menggunakan kode berikut. Kita sudah pernah melakukan hal yang sama di pengenalan DataFrame Spark.

In [ ]:
flightsCleanData.describe('DepDelay','ArrDelay').show()

Selanjutnya kita dapat menghitung nilai korelasi antar variabel delay keberangkatan (Depelay) dan delay pesawat tiba (ArrDelay). Seperti yang kita telah diskusikan di teori data mining di awal buku ini, jika antara dua variable memiliki keterkaitan/pengaruh yang kuat, maka nilai korelasinya akan mendekati 1 (-1 untuk korelasi negatif), sedangkan jika tidak saling terkait, maka nilanya akan 0. Berikut ini adalah kodenya.

In [ ]:
korelasiDelay = flightsCleanData.corr('DepDelay', 'ArrDelay')
print("korelasi antara delay keberangkatan dan delay pesawat tiba: ", 
      korelasiDelay)

Dari nilai di atas, dapat kita simpulkan bahwa delay keberangkatan akan sangat berpengaruh terhadap delay pesawat tiba. Hal ini juga logis, bahwa ketika pesawat delay berangkat, maka kemungkinan besar pesawat akan delay saat tiba di bandara tujuan. Kita juga melihat bahwa saat mengeksplorasi data ini kita menggunakan DataFrame "flightsCleanData", yaitu data bersih yang sudah kita preprocessing sebelumya. Adapun saat melakukan teknik-teknik data mining nanti, kita juga akan menggunakan data yang sudah kita preprocessing.